In [163]:
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
import random
from feature_engine.imputation import MeanMedianImputer
from sklearn.cluster import KMeans

In [817]:
project_name = 'Redisson'
commit_project_name = 'redisson'

In [818]:
okhttp_commits_df = pd.read_csv(f'raw_commits_github_json/{project_name}/{commit_project_name}_commits.csv')

In [819]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,e7afa520697f140bb988216c32a86222082c565e,Nikita Koksharov,mrniko@users.noreply.github.com,2021-11-03 07:21:28,mrniko,Merge pull request #3930 from deerRule/master\...,ae889ad268e3504d027e8edf2f1f8b09483c226c,NaN,NaN,0
1,75b37cdd63d881150de4a6b60f07c92ed1473b7e,xulu,deerrule@qq.com,2021-11-02 10:56:15,deerRule,refactoring\n\nSigned-off-by: xulu <deerrule@q...,478371b8549fafa4f8cb40a9f2be3dac46b2e1fe,2021-11-02 09:43:44,1.0,0
2,478371b8549fafa4f8cb40a9f2be3dac46b2e1fe,deerRule,39396368+deerRule@users.noreply.github.com,2021-11-02 09:43:44,deerRule,Merge pull request #1 from redisson/master\n\n...,796645f655e348fc782d68ce89fd69841fc92ca7,2021-11-02 09:02:17,0.0,0
3,796645f655e348fc782d68ce89fd69841fc92ca7,xulu,deerrule@qq.com,2021-11-02 09:02:17,deerRule,fix: fix bug for msetnx cross slot in Redis cl...,fa82ffa2b0b278057411c08d31fc12a61396c0b3,2021-10-29 09:33:34,95.0,0
4,fa82ffa2b0b278057411c08d31fc12a61396c0b3,Nikita Koksharov,mrniko@users.noreply.github.com,2021-10-29 09:33:34,mrniko,Update README.md,27f0b602935d99b054a89c65433ca582abf37901,2021-10-29 09:32:49,0.0,0


In [820]:
len(okhttp_commits_df)

1610

In [821]:
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] > 0]
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] < 168]
len(okhttp_commits_df)

306

In [822]:
okhttp_commits_df['message'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [823]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 32 mins
H2O_cluster_timezone:,Asia/Singapore
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.7
H2O_cluster_version_age:,2 months and 12 days
H2O_cluster_name:,H2O_from_python_tanji_zpsor2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.886 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [824]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [825]:
STOP_WORDS = stopwords.words('english')
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]
STOP_WORDS = ['']



In [826]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [827]:
def create_token_table(data_hdf, col_name, prefix, sent_sample_rate = 0.0, epochs = 25, vec_size=20):
    # Tokenize
    words = tokenize(data_hdf[col_name])
    w2v_model = H2OWord2vecEstimator(sent_sample_rate = sent_sample_rate, 
                                     epochs = epochs, 
                                     vec_size = vec_size)
    w2v_model.train(training_frame=words)
    
    # Create the training vectors
    vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
    
    
    # Prefix naming the columns
    vecs.columns = ['%s%s' % (prefix, c) for c in vecs.columns]
    data2_hdf = data_hdf.cbind(vecs)
    return data2_hdf, w2v_model

    
#     data3_hdf = data2_hdf[mask,:].cbind(vecs[mask,:])

In [828]:
commit_message_df = okhttp_commits_df[['message',]]

In [829]:
commit_message_df.shape

(306, 1)

In [830]:
commit_message_df.head()

,message
1,refactoring\n\nSigned-off-by: xulu <deerrule@q...
3,fix: fix bug for msetnx cross slot in Redis cl...
28,Update CHANGELOG.md
31,Merge pull request #3826 from Cesarla/2714__fi...
34,fix: fix bug for mset cross slot (miss values ...


In [831]:
commit_message_hdf = h2o.H2OFrame(commit_message_df, 
                         column_types = {'message': 'string'}
                        )
commit_message_hdf.types

Parse progress: |█████████████████████████████████████████████████████████| 100%


{'message': 'string'}

In [832]:
commit_message_hdf.head()

message
refactoring Signed-off-by: xulu <deerrule@qq.com>
fix: fix bug for msetnx cross slot in Redis cluster mode Signed-off-by: xulu <deerrule@qq.com>
Update CHANGELOG.md
"Merge pull request #3826 from Cesarla/2714__fix_try_lock_interrupted_keeps_renewing_lock_indefinitely Fix ""TryLock interrupted keeps renewing lock indefinitely"""
fix: fix bug for mset cross slot (miss values except cluster mode) Signed-off-by: xujie <mikawudi@qq.com>
Update README.md
Update CHANGELOG.md
Merge pull request #2995 from dengliming/f_calcslot_bug Fix ClusterConnectionManager#calcSlot bug.
style(starter): remain only changed lines remain only changed lines Signed-off-by: <anjia0532@gmail.com>
Merge pull request #3000 from sulake/bugfix/write_mode Write mode for modifying set.


In [833]:
# Train owner names model
commit_message_vecs_hdf, commit_message_w2v_model = create_token_table(commit_message_hdf,
                                                                  'message',
                                                                   prefix='message_',
                                                                   vec_size=30)

'''
words = tokenize(commit_message_hdf['message'])
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, 
                                 vec_size = 30)
w2v_model.train(training_frame=words)

# Create the training vectors
vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")


# Prefix naming the columns
vecs.columns = ['%s%s' % ('message_', c) for c in vecs.columns]
data2_hdf = commit_message_hdf.cbind(vecs)
print(data2_hdf)

wv_path = 'models/'
model_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)

#Load Model
w2v_model2 = h2o.load_model(model_path)
'''

word2vec Model Build progress: |██████████████████████████████████████████| 100%


'\nwords = tokenize(commit_message_hdf[\'message\'])\nw2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, \n                                 vec_size = 30)\nw2v_model.train(training_frame=words)\n\n# Create the training vectors\nvecs = w2v_model.transform(words, aggregate_method = "AVERAGE")\n\n\n# Prefix naming the columns\nvecs.columns = [\'%s%s\' % (\'message_\', c) for c in vecs.columns]\ndata2_hdf = commit_message_hdf.cbind(vecs)\nprint(data2_hdf)\n\nwv_path = \'models/\'\nmodel_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)\n\n#Load Model\nw2v_model2 = h2o.load_model(model_path)\n'

In [834]:
commit_message_w2v_model.find_synonyms('refactor')

OrderedDict([('refactor', 1.0),
             ('comments', 0.7956967949867249),
             ('xujie', 0.7921608090400696),
             ('remove', 0.7846896052360535),
             ('gmail', 0.7767580151557922),
             ('tom', 0.7566903829574585),
             ('qq', 0.7477058172225952),
             ('by', 0.7334879040718079),
             ('stwer', 0.7150746583938599),
             ('erik', 0.7140863537788391),
             ('off', 0.7089080214500427),
             ('methods', 0.6991560459136963),
             ('mikawudi', 0.6818760633468628),
             ('change', 0.6300732493400574),
             ('johno', 0.6129664778709412),
             ('fixed', 0.6029987335205078),
             ('all', 0.5930441617965698),
             ('rename', 0.5812867283821106),
             ('some', 0.5743890404701233),
             ('tobias', 0.5678851008415222)])

In [835]:
rand_list = sorted( random.sample(range(commit_message_vecs_hdf.nrow), 4 ) )
commit_message_vecs_hdf[rand_list, :]

message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,message_C10,message_C11,message_C12,message_C13,message_C14,message_C15,message_C16,message_C17,message_C18,message_C19,message_C20,message_C21,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30
Update README.md,-0.472446,1.26229,0.0679948,0.0207544,-0.781682,-0.243154,0.590302,-1.51023,0.287612,0.0736333,1.21139,-0.366389,-0.506921,0.58641,-0.104019,-0.0904669,-0.13987,-0.158792,0.878411,0.46333,-0.568447,-0.726011,-0.11872,0.816537,0.713669,-0.555292,0.178028,-0.542236,0.145823,-0.195545
removed unnecessary creation of cron expression,-0.141495,0.560893,-0.280749,-0.179739,0.0244038,0.0486227,0.255634,0.0324873,0.288384,0.0467491,0.259575,-0.186376,-0.525591,0.139532,-0.0566814,-0.296069,-0.145873,0.470408,-0.550656,-0.169302,-0.353118,-0.00081683,0.316304,-0.256023,-0.105614,-0.313239,0.111726,-0.0370711,-0.175778,-0.189699
Update README.md,-0.472446,1.26229,0.0679948,0.0207544,-0.781682,-0.243154,0.590302,-1.51023,0.287612,0.0736333,1.21139,-0.366389,-0.506921,0.58641,-0.104019,-0.0904669,-0.13987,-0.158792,0.878411,0.46333,-0.568447,-0.726011,-0.11872,0.816537,0.713669,-0.555292,0.178028,-0.542236,0.145823,-0.195545
Update CHANGELOG.md,-0.543001,1.21016,0.0988275,-0.0522884,-0.762135,-0.219013,0.522927,-1.51801,0.286342,0.0955643,1.17313,-0.490607,-0.48575,0.622013,-0.192305,-0.0979018,-0.200668,-0.258089,0.933855,0.520266,-0.536337,-0.59875,-0.130141,0.959933,0.769633,-0.574485,0.0955345,-0.592163,0.0977756,-0.323097


In [836]:
kmean = KMeans(n_clusters=15)

In [837]:
commit_message_vecs_hdf = commit_message_vecs_hdf.as_data_frame()

In [838]:
commit_message_vecs_hdf = commit_message_vecs_hdf.fillna(0)

In [839]:
commit_message_vecs_hdf.iloc[:,1:]

,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,message_C10,...,message_C21,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30
0,0.298063,0.065312,-0.168075,0.032445,0.269284,0.045872,0.359772,-0.178879,0.074347,0.491671,...,-0.725633,0.366066,0.134145,0.133491,0.967539,0.252309,-0.252676,0.032904,-0.795650,-0.658313
1,0.275907,0.251152,-0.129204,0.110814,0.325148,-0.141281,0.232259,-0.199494,0.268469,0.220535,...,-0.490642,-0.104047,0.180858,-0.026988,0.485131,0.161349,-0.032794,0.041812,-0.227148,-0.209902
2,-0.543001,1.210164,0.098828,-0.052288,-0.762135,-0.219013,0.522927,-1.518010,0.286342,0.095564,...,-0.536337,-0.598750,-0.130141,0.959933,0.769633,-0.574485,0.095534,-0.592163,0.097776,-0.323097
3,0.011228,0.034626,-0.525885,0.174363,-0.040151,-0.031499,0.036773,0.288774,0.642305,0.087541,...,-0.426918,-0.654577,0.415498,-0.081584,0.277492,-0.050067,0.078594,0.572818,0.207372,0.076710
4,0.341388,0.190126,-0.155601,0.037204,0.273973,0.020312,0.287848,-0.236800,0.344122,0.321978,...,-0.528884,0.034273,0.269396,0.062054,0.701722,0.194614,-0.080835,0.103659,-0.416159,-0.385434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0.269839,0.053393,-0.202248,0.020207,0.291285,-0.307342,0.105650,-0.088124,0.127603,0.407192,...,-0.510877,0.372588,0.067184,0.043335,0.909810,0.198033,-0.030303,0.196892,-0.440957,-0.599431
302,0.265929,0.408993,-0.454413,-0.039036,-0.052785,0.024657,0.193808,0.153812,0.194320,0.331016,...,-0.617125,0.203009,0.165812,0.215122,0.253051,0.027214,-0.041339,0.357943,-0.444443,-0.504487
303,0.370786,0.190095,-0.073957,0.078649,0.269690,-0.345854,0.121252,-0.143224,0.220990,0.308591,...,-0.453886,0.062330,0.075069,0.001016,0.541723,0.196066,-0.001291,0.039646,-0.228543,-0.284559
304,0.309816,0.182765,-0.199913,0.062888,0.310961,-0.210750,0.162494,-0.141580,0.199706,0.437972,...,-0.601086,0.198756,0.114258,0.163648,0.696110,0.199267,-0.104974,0.074190,-0.418139,-0.444421


In [840]:
kmean.fit(commit_message_vecs_hdf.iloc[:,1:])

KMeans(n_clusters=15)

In [841]:
kmean.labels_

array([ 0, 10,  1,  3, 10,  1,  1,  3,  0,  3,  3,  0,  0,  0, 11, 11, 11,
       12,  0,  1,  8,  8,  8, 10,  1,  3, 10,  1,  3,  1,  1,  1,  3, 10,
        1, 10,  9, 14, 14, 10, 10,  4,  1,  1,  1,  1,  3,  3,  1,  3, 10,
        1,  3,  1,  1,  4,  1,  1,  1,  3,  9,  4,  1,  1,  1,  1,  4,  1,
        1,  1,  1, 11,  1,  1,  1, 14,  4,  4,  6,  1,  3,  4, 12,  7,  3,
        2, 14,  1,  1,  3, 10,  1, 14,  1,  1,  1,  1,  7,  1,  1,  1,  1,
        3,  6,  9, 12, 14,  3,  3,  3,  1,  1,  1,  7,  6,  1,  7, 14, 11,
        1,  3,  0,  0,  0,  0,  0, 10,  1,  1, 14,  7,  2,  2,  5,  9,  4,
        3, 10,  4,  6,  6,  3,  1,  3, 11,  1,  3,  1,  1,  9,  9,  3, 14,
        6, 13,  4,  1,  3,  4,  1, 13,  3,  9,  1,  3,  1,  1,  1,  1,  1,
        1,  1,  3,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  3,  5,  6,  5,  4,  4,  6,  6,  3,  7,  7,  7,  6, 13,  3,
        1,  1,  1,  1,  1,  1, 10, 10, 10,  0,  0,  0,  0, 11,  6,  5,  7,
       10,  6,  6,  6, 12

In [842]:
print(len(kmean.labels_))

306


In [843]:
commit_message_vecs_hdf['kmean_label'] = kmean.labels_

In [844]:
commit_message_vecs_hdf.head()

,message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,...,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30,kmean_label
0,refactoring\r\n\r\nSigned-off-by: xulu <deerru...,0.298063,0.065312,-0.168075,0.032445,0.269284,0.045872,0.359772,-0.178879,0.074347,...,0.366066,0.134145,0.133491,0.967539,0.252309,-0.252676,0.032904,-0.795650,-0.658313,0
1,fix: fix bug for msetnx cross slot in Redis cl...,0.275907,0.251152,-0.129204,0.110814,0.325148,-0.141281,0.232259,-0.199494,0.268469,...,-0.104047,0.180858,-0.026988,0.485131,0.161349,-0.032794,0.041812,-0.227148,-0.209902,10
2,Update CHANGELOG.md,-0.543001,1.210164,0.098828,-0.052288,-0.762135,-0.219013,0.522927,-1.518010,0.286342,...,-0.598750,-0.130141,0.959933,0.769633,-0.574485,0.095534,-0.592163,0.097776,-0.323097,1
3,Merge pull request #3826 from Cesarla/2714__fi...,0.011228,0.034626,-0.525885,0.174363,-0.040151,-0.031499,0.036773,0.288774,0.642305,...,-0.654577,0.415498,-0.081584,0.277492,-0.050067,0.078594,0.572818,0.207372,0.076710,3
4,fix: fix bug for mset cross slot (miss values ...,0.341388,0.190126,-0.155601,0.037204,0.273973,0.020312,0.287848,-0.236800,0.344122,...,0.034273,0.269396,0.062054,0.701722,0.194614,-0.080835,0.103659,-0.416159,-0.385434,10


In [845]:
group_list = sorted(commit_message_vecs_hdf.kmean_label.unique())
for group_id in group_list:
    mask = commit_message_vecs_hdf.kmean_label == group_id
    print('group_id = %d' % group_id)
    min_size = min(sum(mask), 10)
    print( commit_message_vecs_hdf[mask].sample(min_size) )

group_id = 0
                                               message  message_C1  \
124  Rename counter\r\n\r\nSigned-off-by: Tom Erik ...    0.218933   
300  RedisNodes ping results check optimization\r\n...    0.220492   
11   Add batch support for revRank. Simplify naming...    0.432358   
12   Use IntegerReplayConvertor.\r\n\r\nSigned-off-...    0.521913   
121  fix: release ByteBuf causing leak. solves #356...    0.149172   
122  Call hget only when needed\r\n\r\nSigned-off-b...    0.151755   
216  fix: ClientConnectionsEntry reconnect error\r\...    0.313536   
125  Rename methods\r\n\r\nSigned-off-by: Tom Erik ...    0.229614   
0    refactoring\r\n\r\nSigned-off-by: xulu <deerru...    0.298063   
214  refactor: remove some empty lines\r\n\r\nSigne...    0.291915   

     message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
124   -0.108458   -0.172375    0.199433    0.417161    0.110989    0.253954   
300    0.057585   -0.196919    0.029931    0.328982   -0.1

[6 rows x 32 columns]
group_id = 3
                                               message  message_C1  \
172  Merge pull request #748 from jribble/local_cac...    0.120724   
107  Merge pull request #1051 from sdjacobs/inherit...    0.025112   
102  Merge pull request #1124 from joseduraes/fix-m...    0.181000   
7    Merge pull request #2995 from dengliming/f_cal...    0.066806   
109  Merge pull request #968 from Vcgoyo/master\r\n...   -0.021140   
141  Merge pull request #832 from serkan-ozal/overr...   -0.064104   
164  log error when remote service can't process th...    0.008608   
3    Merge pull request #3826 from Cesarla/2714__fi...    0.011228   
161  Merge pull request #777 from jribble/SoftCache...    0.162393   
9    Merge pull request #3000 from sulake/bugfix/wr...    0.103261   

     message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
172    0.009959   -0.403364    0.167210   -0.139120   -0.068188    0.053581   
107    0.106132   -0.221026    0.128

[10 rows x 32 columns]
group_id = 6
                                               message  message_C1  \
244  The red test case (about executorsAmount that ...    0.212369   
271  added all the redis v3.0.7 config options as e...    0.404083   
238  WIP\r\n\r\n1. Simplified the API\r\n2. Added c...    0.168066   
255  Tidy up\r\nRemoved unused imports from tests\r...    0.160134   
114  moved parseAttribute method to parser support ...    0.032942   
269  Completed RedisRunner mods\r\n\r\n* RedisRunne...    0.239872   
153       XSD typo fixed and added a missing attribute    0.059339   
267  added sudo false\r\n\r\nWe are going to the co...    0.172660   
257                    set maximum heap size for tests    0.364311   
140        checks and fixed a few more different usage   -0.012176   

     message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
244    0.410681    0.186674   -0.014827    0.137282   -0.395314    0.454293   
271    0.223411    0.122928    0.12

[3 rows x 32 columns]
group_id = 9
                                               message  message_C1  \
104  Better collection handling for RedissonReferen...    0.462316   
60   redisson#2007: Lookup a channel's codec correc...    0.340467   
275  uses pom property for source code version refe...    0.234570   
150  Add test coverage for org.redisson.codec.AvroJ...    0.451577   
162  Add an option to use soft references for evict...    0.258110   
134                            add unit tests for #843    0.492726   
149  Merge pull request #800 from marlonwc3/tests/a...    0.371196   
247  Tidy up\r\n1. Rename TestNameToConsoleRunListe...    0.329141   
228  Adding keySize functionality for MultiMaps - I...    0.462316   
229  fixed a wrong reference for codec provider in ...    0.162675   

     message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
104    0.403175    0.075144    0.089888    0.173740   -0.368280    0.003203   
60     0.408435    0.044053    0.136

[7 rows x 32 columns]
group_id = 12
                                               message  message_C1  \
225      Merge branch 'mrniko/master' into live-object    0.024211   
288                       Merge branch 'mrniko/master'    0.160751   
17   Merge remote-tracking branch 'upstream/master'...    0.252550   
274                       Merge branch 'mrniko/master'    0.160751   
285                       Merge branch 'mrniko/master'    0.160751   
82   Merge remote-tracking branch 'upstream/master'...    0.180685   
231      Merge branch 'mrniko/master' into live-object    0.024211   
276                       Merge branch 'mrniko/master'    0.160751   
272  Merge remote-tracking branch 'mrniko/master' i...    0.184212   
268       Merge remote-tracking branch 'mrniko/master'    0.194442   

     message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
225    0.451612   -0.674869   -0.291806   -0.244426    0.127059    0.255943   
288    0.576862   -0.932710   -0.36

[10 rows x 32 columns]


In [846]:
print(len(okhttp_commits_df))
print(len(commit_message_vecs_hdf))

306
306


In [847]:
okhttp_commits_df['kmean_label'] = kmean.labels_
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
1,75b37cdd63d881150de4a6b60f07c92ed1473b7e,xulu,deerrule@qq.com,2021-11-02 10:56:15,deerRule,refactoring\n\nSigned-off-by: xulu <deerrule@q...,478371b8549fafa4f8cb40a9f2be3dac46b2e1fe,2021-11-02 09:43:44,1.0,0,0
3,796645f655e348fc782d68ce89fd69841fc92ca7,xulu,deerrule@qq.com,2021-11-02 09:02:17,deerRule,fix: fix bug for msetnx cross slot in Redis cl...,fa82ffa2b0b278057411c08d31fc12a61396c0b3,2021-10-29 09:33:34,95.0,0,10
28,505778a37f5b8c783b4e87fefa6bf85b37e90a54,Nikita Koksharov,mrniko@users.noreply.github.com,2021-09-21 10:13:16,mrniko,Update CHANGELOG.md,69d7c4fd0085993772b7dd412e46c2bf31c37ae8,2021-09-21 08:54:24,1.0,0,1
31,4e10035e94bf489b6eb0fb1ca46f6d80327bfa71,Nikita Koksharov,mrniko@users.noreply.github.com,2021-09-13 06:22:46,mrniko,Merge pull request #3826 from Cesarla/2714__fi...,21e0c18dcd237c0cca844beaeb550caeff15f4c8,2021-09-10 13:01:26,65.0,0,3
34,eedd7b0f15d95944c2328b42e79dd9f3175d05e5,xujie,mikawudi@qq.com,2021-09-07 12:39:09,mikawudi,fix: fix bug for mset cross slot (miss values ...,cb162a9b25eb41d89880483e279dc0f3a0245e71,2021-09-07 08:41:50,3.0,0,10


In [848]:
okhttp_commits_df['kmean_label'].value_counts()

1     89
3     35
10    29
6     29
14    21
4     19
0     16
5     14
12    12
9     11
7     11
11     7
2      6
13     4
8      3
Name: kmean_label, dtype: int64

In [849]:
okhttp_commits_df['kmean_label'].isna().sum()

0

In [850]:
okhttp_commits_df[okhttp_commits_df['contains_refactoring'] == 1]

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
1152,d1eb2ece8f9ddb4cc797fd236cc4462682372ef8,jackygurui,jackygurui@gmail.com,2016-09-08 22:59:19,jackygurui,Renamed ExpirableInterceptor to RExpirableInte...,552eb98012897cdefd1a70231eef040e1ee6abe9,2016-09-08 21:11:34,1.0,1,7
1443,2c24a344a3f8d7e8e9ebd5649237d68844e34a2f,Vorotyntsev,vorotyntsevdanila@gmail.com,2021-01-24 19:26:54,Vorotyntsev,Fixed according to comments\n\nSigned-off-by: ...,996a545ece6dba3b5a42e23eae43b4d985bda71e,2021-01-22 14:36:45,52.0,1,10


In [851]:
grouped_main_commits = okhttp_commits_df.groupby(['kmean_label', 'login']).agg({'time_taken': ['mean', 'min', 'max']})['time_taken'].reset_index()
grouped_main_commits

,kmean_label,login,mean,min,max
0,0,anjia0532,119.0,119.0,119.0
1,0,blackstorm,18.0,18.0,18.0
2,0,deerRule,1.0,1.0,1.0
3,0,johnou,20.5,14.0,35.0
4,0,mikawudi,29.0,5.0,90.0
...,...,...,...,...,...
76,14,pepelu0,77.0,77.0,77.0
77,14,pierredavidbelanger,17.5,12.0,23.0
78,14,scubasau,19.0,19.0,19.0
79,14,sdjacobs,12.0,12.0,12.0


In [852]:
grouped_main_commits[grouped_main_commits['kmean_label']==0]

,kmean_label,login,mean,min,max
0,0,anjia0532,119.0,119.0,119.0
1,0,blackstorm,18.0,18.0,18.0
2,0,deerRule,1.0,1.0,1.0
3,0,johnou,20.5,14.0,35.0
4,0,mikawudi,29.0,5.0,90.0
5,0,testower,25.6,1.0,71.0


In [853]:
okhttp_commits_df_refactored = okhttp_commits_df[okhttp_commits_df['contains_refactoring'] == 1]
okhttp_commits_df_refactored.groupby(['kmean_label','login']).agg({'time_taken': ['mean', 'min', 'max']})

time_taken            
                              mean   min   max
kmean_label login                             
7           jackygurui         1.0   1.0   1.0
10          Vorotyntsev       52.0  52.0  52.0

In [854]:
final_df = pd.merge(okhttp_commits_df_refactored, grouped_main_commits, how='left', on=['login','kmean_label'])

In [855]:
final_df['project_name'] = project_name

In [856]:
final_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label,mean,min,max,project_name
0,d1eb2ece8f9ddb4cc797fd236cc4462682372ef8,jackygurui,jackygurui@gmail.com,2016-09-08 22:59:19,jackygurui,Renamed ExpirableInterceptor to RExpirableInte...,552eb98012897cdefd1a70231eef040e1ee6abe9,2016-09-08 21:11:34,1.0,1,7,12.0,1.0,44.0,Redisson
1,2c24a344a3f8d7e8e9ebd5649237d68844e34a2f,Vorotyntsev,vorotyntsevdanila@gmail.com,2021-01-24 19:26:54,Vorotyntsev,Fixed according to comments\n\nSigned-off-by: ...,996a545ece6dba3b5a42e23eae43b4d985bda71e,2021-01-22 14:36:45,52.0,1,10,29.0,19.0,52.0,Redisson


In [857]:
project_refactoring_data = pd.read_csv(f'Effort_Estimation_Results_v2/{project_name}.csv')

In [858]:
project_refactoring_data.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,modifiers,latest_commit,commit_compared_with,num_dependency,num_line_affected,actual_num_of_classes_touched,latest_commit_date,commit_compared_with_date,author,author_email
0,E:\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.config.RedissonNodeFileConfig,class,4,11,2,2,37,11,2,...,1,ed52098cb4a2e5d854aeef703d90aa8a1186c156,383237229a71e738795f6ad58a3ad51a88ae9be5,8.0,100,3,2021-06-07 07:46:55,2021-05-19 09:30:30,Nikita Koksharov,nkoksharov@redisson.pro
1,E:\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.micronaut.cache.BaseCacheConfigur...,class,8,16,1,6,80,16,0,...,1,592e88966590e45f8b67cb34dccd41689a00c8cf,588669d62fad7cc351490d614053e8ec559c5765,331.0,140,1,2021-06-28 08:29:12,2021-06-21 06:53:17,Nikita Koksharov,nkoksharov@redisson.pro
2,E:\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.client.protocol.decoder.ListFirst...,class,3,3,1,2,1,2,0,...,1,db0bbcca0abd8d5eec57552a5d8729d858c99276,d45af58259be846bc46b7ffee281caf3456fbe86,8.0,20,1,2019-11-28 10:08:36,2016-12-17 17:58:28,Nikita,abracham.mitchell@gmail.com
3,E:\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.api.CronSchedule,class,2,9,1,2,0,6,4,...,17,db0bbcca0abd8d5eec57552a5d8729d858c99276,1f1f0b7bc2e0c9acf193d6d5b281b1959287d360,14.0,86,1,2019-11-28 10:08:36,2016-12-23 11:08:18,Nikita,abracham.mitchell@gmail.com
4,E:\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.eviction.EvictionScheduler,class,6,10,1,3,0,6,0,...,1,db0bbcca0abd8d5eec57552a5d8729d858c99276,642a605b0ab33d3d3dd12bd3901a2423acc0970e,110.0,50,1,2019-11-28 10:08:36,2017-02-01 12:25:39,Nikita,abracham.mitchell@gmail.com


In [859]:
project_refactoring_data.columns

Index(['file', 'class', 'type', 'cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers', 'latest_commit', 'commit_compared_with', 'num_dependency',
       'num_line_affected', 'actual_num_of_classes_touched',
       'latest_commit_date', 'commit_compared_with_date', 'author',
       'author_email'],
      dtype='object')

In [860]:
columns_to_agg = [
    'cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers', 'num_dependency',
       'num_line_affected', 'actual_num_of_classes_touched'
]


project_refactoring_data_agg_column_names = ['commit_compared_with']
for element in columns_to_agg:
    project_refactoring_data_agg_column_names.append(element + '_mean')
    project_refactoring_data_agg_column_names.append(element + '_min')
    project_refactoring_data_agg_column_names.append(element + '_max')
    project_refactoring_data_agg_column_names.append(element + '_std')


In [861]:
project_refactoring_data_agg = project_refactoring_data.groupby(['commit_compared_with'], as_index=False).agg({
       'cbo': ['mean', 'min', 'max','std'], 
        'wmc': ['mean', 'min', 'max','std'], 
        'dit': ['mean', 'min', 'max','std'], 
        'rfc': ['mean', 'min', 'max','std'], 
        'lcom': ['mean', 'min', 'max','std'],
        'totalMethods': ['mean', 'min', 'max','std'], 
        'staticMethods': ['mean', 'min', 'max','std'],
        'publicMethods': ['mean', 'min', 'max','std'], 
        'privateMethods': ['mean', 'min', 'max','std'],
        'protectedMethods': ['mean', 'min', 'max','std'], 
        'defaultMethods': ['mean', 'min', 'max','std'], 
        'abstractMethods': ['mean', 'min', 'max','std'], 
        'finalMethods': ['mean', 'min', 'max','std'],
        'synchronizedMethods': ['mean', 'min', 'max','std'],
        'totalFields': ['mean', 'min', 'max','std'], 
        'staticFields': ['mean', 'min', 'max','std'], 
        'publicFields': ['mean', 'min', 'max','std'],
        'privateFields': ['mean', 'min', 'max','std'],
        'protectedFields': ['mean', 'min', 'max','std'],
        'defaultFields': ['mean', 'min', 'max','std'], 
        'finalFields': ['mean', 'min', 'max','std'],
        'synchronizedFields': ['mean', 'min', 'max','std'],
        'nosi': ['mean', 'min', 'max','std'], 
        'loc': ['mean', 'min', 'max','std'], 
        'returnQty': ['mean', 'min', 'max','std'],
        'loopQty': ['mean', 'min', 'max','std'],
        'comparisonsQty': ['mean', 'min', 'max','std'],
        'tryCatchQty': ['mean', 'min', 'max','std'],
        'parenthesizedExpsQty': ['mean', 'min', 'max','std'],
        'stringLiteralsQty': ['mean', 'min', 'max','std'],
        'numbersQty': ['mean', 'min', 'max','std'], 
        'assignmentsQty': ['mean', 'min', 'max','std'],
        'mathOperationsQty': ['mean', 'min', 'max','std'], 
        'variablesQty': ['mean', 'min', 'max','std'], 
        'maxNestedBlocks': ['mean', 'min', 'max','std'],
        'anonymousClassesQty': ['mean', 'min', 'max','std'],
        'subClassesQty': ['mean', 'min', 'max','std'],
        'lambdasQty': ['mean', 'min', 'max','std'],
        'uniqueWordsQty': ['mean', 'min', 'max','std'],
        'modifiers': ['mean', 'min', 'max','std'],
        'num_dependency': ['mean', 'min', 'max','std'],
        'num_line_affected': ['mean', 'min', 'max','std'],
        'actual_num_of_classes_touched': ['mean', 'min', 'max','std']
})


In [862]:
project_refactoring_data_agg.columns = project_refactoring_data_agg_column_names

In [863]:
project_refactoring_data_agg.head()

,commit_compared_with,cbo_mean,cbo_min,cbo_max,cbo_std,wmc_mean,wmc_min,wmc_max,wmc_std,dit_mean,...,num_dependency_max,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std,actual_num_of_classes_touched_mean,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std
0,01012972bfccb29a442a8d4f8bc4c4a06df8b244,16.000000,16,16,NaN,35.000000,35,35,NaN,1.000000,...,21.0,NaN,256.000000,256,256,NaN,31.000000,31,31,NaN
1,012d2e7fbb69ae5a70fe48b2653a96849330381e,2.000000,2,2,NaN,1.000000,1,1,NaN,1.000000,...,37.0,NaN,18.000000,18,18,NaN,35.000000,35,35,NaN
2,044ebebbf2a5992a1e8aa8b9e799ed31696fd673,19.000000,19,19,NaN,23.000000,23,23,NaN,2.000000,...,28.0,NaN,149.000000,149,149,NaN,9.000000,9,9,NaN
3,07131b883a346e036f0fb6087b8e1fa93affec84,7.000000,7,7,NaN,12.000000,12,12,NaN,2.000000,...,104.0,NaN,161.000000,161,161,NaN,11.000000,11,11,NaN
4,07ae51add516aed88d193a8c986e17f0531fc82b,4.857143,2,10,3.023716,14.285714,1,52,17.820534,2.285714,...,373.0,133.303449,101.857143,9,441,155.679616,4.285714,1,9,2.690371


In [864]:
final_df_agg = pd.merge(final_df, project_refactoring_data_agg, how='inner', left_on='sha', right_on='commit_compared_with')

In [865]:
final_df_agg.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,...,num_dependency_max,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std,actual_num_of_classes_touched_mean,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std
0,d1eb2ece8f9ddb4cc797fd236cc4462682372ef8,jackygurui,jackygurui@gmail.com,2016-09-08 22:59:19,jackygurui,Renamed ExpirableInterceptor to RExpirableInte...,552eb98012897cdefd1a70231eef040e1ee6abe9,2016-09-08 21:11:34,1.0,1,...,42.0,NaN,18.0,18,18,NaN,1.0,1,1,NaN
1,2c24a344a3f8d7e8e9ebd5649237d68844e34a2f,Vorotyntsev,vorotyntsevdanila@gmail.com,2021-01-24 19:26:54,Vorotyntsev,Fixed according to comments\n\nSigned-off-by: ...,996a545ece6dba3b5a42e23eae43b4d985bda71e,2021-01-22 14:36:45,52.0,1,...,104.0,NaN,30.0,30,30,NaN,7.0,7,7,NaN


In [866]:
len(final_df)

2

In [867]:
len(final_df_agg)

2

In [868]:
final_df_agg.to_csv(f'Effort_Estimation_Results_v3/{project_name}.csv', index=False)